# Demo - Constraint Based Synthesis

Dependencies
<!-- - OPTIONAL: [Docker](https://www.docker.com/) -->
- Python 3ish
- [Jupyter](https://jupyter.org/)
- [Racket](https://racket-lang.org/)
- [Rosette](https://docs.racket-lang.org/rosette-guide/index.html): constraint based solver based on racket

Setup (No Docker)
1. Download & install [Racket](https://racket-lang.org/), add to path
2. Run:
    ```sh
    # optional but recommended: virtualenv
    python3 -m venv .venv
    source .venv/bin/activate
    pip install -r requirements.txt  # this installs jupyter

    # install rosette
    raco pkg install rosette

    jupyter notebook
    ```
3. Change environment variables at the top of your target notebook as required, as you may need to add racket to the venv path manually

In [2]:
# Julia -- this is for my machine
import os

# Add /Applications/Racket v8.16/bin to PATH
os.environ['PATH'] = '/Applications/Racket v8.16/bin:$PATH'
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib:$DYLD_LIBRARY_PATH'

#todos

Dan --> linear function with examples, verbose mode

Julia --> simple arithmetic DSL with `define-grammar` & sketching

# Example 1
## _Linear Function Synthesis from Examples_

Find a function of the form $f(x) = Ax + B$ that satisfies the following examples: 
- $f(1) = 5$
- $f(2) = 7$
- $f(3) = 9$
- $f(4) = 11$

---
```racket
#lang rosette
(require rosette/lib/synthax) ; the package containing `synthesis` syntax

; Define a function `f` with unknown coefficients `A` and `B`
(define (f x)
  (+ (* (?? integer?) x) (?? integer?)))  ; `??` means a hole in the program

; Examples (x . y), meaning f(x) = y
(define examples
  `((1 . 5)  (2 . 7)  (3 . 9)  (4 . 11)))

; Synthesis constraint: f(x) must match expected output
(define sol
  (synthesize
   #:forall (list)
   #:guarantee
   (begin
     (for/list ([pair examples])
       (assert (= (f (car pair)) (cdr pair)))))))

(print-forms sol)
```


In [3]:
!racket racket/linear-func-synth.rkt

/Users/julianonn/pv703/cs703-rosette/racket/linear-func-synth.rkt:9:0
(define (f x) (+ (* 2 x) 3))


__Example 1 - Result:__   $f(x) = 2x + 3$

In [4]:
# Example 1 -- Verbose Mode
!racket racket/VERBOSE-linear-func-synth.rkt

(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :smt.relevancy 2)
(declare-fun c0 () Int)
(declare-fun c1 () Int)
(define-fun e2 () Int (+ c0 c1))
(define-fun e3 () Bool (= 5 e2))
(define-fun e4 () Int (* 2 c0))
(define-fun e5 () Int (+ c1 e4))
(define-fun e6 () Bool (= 7 e5))
(define-fun e7 () Bool (and e3 e6))
(define-fun e8 () Int (* 3 c0))
(define-fun e9 () Int (+ c1 e8))
(define-fun e10 () Bool (= 9 e9))
(define-fun e11 () Bool (and e7 e10))
(define-fun e12 () Int (* 4 c0))
(define-fun e13 () Int (+ c1 e12))
(define-fun e14 () Bool (= 11 e13))
(define-fun e15 () Bool (and e11 e14))
(assert e15)
(check-sat)
(get-model)
(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :smt.relevancy 2)
(reset)
(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :

---
---

# Example 2 

Let's create a DSL for simple arithmetic over integers.

In [11]:
!racket racket/arith-dsl-1.rkt

/Users/julianonn/pv703/cs703-rosette/racket/arith-dsl-1.rkt:21:0
(define (f x) (Add (Mult 2 x) 3))


In [ ]:
!racket racket/arith-dsl-2-sketch.rkt   # this isn't working. TBD on fix

application: not a procedure;
 expected a procedure that can be applied to arguments
  given: (unsat)
  context...:
   /Users/julianonn/Library/Racket/8.16/pkgs/rosette/rosette/query/eval.rkt:19:0: eval-rec
   /Users/julianonn/Library/Racket/8.16/pkgs/rosette/rosette/query/eval.rkt:65:0: eval-guarded
   /Users/julianonn/Library/Racket/8.16/pkgs/rosette/rosette/query/eval.rkt:19:0: eval-rec
   .../match/compiler.rkt:559:40: f98
   /Users/julianonn/Library/Racket/8.16/pkgs/rosette/rosette/query/eval.rkt:19:0: eval-rec
   body of "/Users/julianonn/pv703/cs703-rosette/racket/arith-dsl-2-sketch.rkt"
